# Fine-tuning BERT (base or large) on a Question-Answering task by using the library adapter-transformers (script version)

- **Credit**: [Hugging Face](https://huggingface.co/) and [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)
- **Author**: [Pierre GUILLOU](https://www.linkedin.com/in/pierreguillou/)
- **Date**: 08/02/2021
- **Blog post**: [NLP nas empresas | Como ajustar um modelo de linguagem natural como BERT para a tarefa de Question-Answering (QA) com um Adapter?]()
- **Link to the folder in github with this notebook and all necessary scripts**: [question-answering with adapters](https://github.com/piegu/language-models/tree/master/adapters/question-answering/)

## 1. Context

### Objective

The objective here is to **fine-tune a Masked Language Model (MLM) like BERT (base or large) for a QA task by training adapters (library [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)), not the embeddings and transformers layers of the MLM model**, and to compare results with BERT model fully fine-tune for the same task.

The interest is obvious: if you need models for different NLP tasks, instead of fine-tuning and storing one model by NLP task, **you store only one MLM model and the trained tasks adapters which sizes are about 1% (QA adapter) and 8% (Lang+QA adapters) of the MLM model one** (it depends of the choosen adapter configuration). More, the loading of these adapters in production is very easy.

### Content

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will use the library [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers) and see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](images/question_answering_adapter.png)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

### History and Credit

This notebook is an adaptation of the following notebooks and scripts for **fine-tuning a (transformer) Masked Language Model (MLM) like BERT (base or large) on the QA task with any QA dataset** (we use here the [Portuguese Squad 1.1 dataset](https://forum.ailab.unb.br/t/datasets-em-portugues/251/4)):
- **from [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)** | notebook [04_Cross_Lingual_Transfer.ipynb](https://github.com/Adapter-Hub/adapter-transformers/blob/master/notebooks/04_Cross_Lingual_Transfer.ipynb) and script [run_qa.py](https://github.com/Adapter-Hub/adapter-transformers/blob/master/examples/question-answering/run_qa.py) (this script was adapted from the script [run_qa.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa.py) of HF)
- **from [transformers](https://github.com/huggingface/transformers) of Hugging Face** | notebook [question_answering.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb) and script [run_qa.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa.py) 

In order to speed up the fine-tuning of the model on only one GPU, the library [DeepSpeed](https://www.deepspeed.ai/) could be used by applying the configuration provided by HF in the notebook [transformers + deepspeed CLI](https://github.com/stas00/porting/blob/master/transformers/deepspeed/DeepSpeed_on_colab_CLI.ipynb) but as the library adapter-transformers is not synchronized with the last version of the library transformers of HF, we keep that option for the future.

### Major changes from original notebooks and scripts

The script [run_qa.py](https://github.com/Adapter-Hub/adapter-transformers/blob/master/examples/question-answering/run_qa.py) allows to evaluate the model performance against f1 metric at the end of each epoch, and not against validation loss as done in the HF notebook [question_answering.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb). This is very important as we consider the metric when selecting a model, not the loss. Therefore, we decided to launch this script inside this notebook ([question_answering_adapter_script.ipynb](https://github.com/piegu/language-models/blob/master/adapters/question-answering/question_answering_adapter_script.ipynb)) (by simulating terminal command line) instead of running code in cells as done in the HF notebook. 

However, to provide an HF-like notebook, we also updated the HF notebook [question_answering.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb) on the notebook [question_answering_adapter.ipynb](https://github.com/piegu/language-models/blob/master/adapters/question-answering/question_answering_adapter.ipynb), but without the f1 metric as the evaluation metric during model training (ie, the f1 metric is used only after the model training).

More, we updated the script [run_qa.py](https://github.com/Adapter-Hub/adapter-transformers/blob/master/examples/question-answering/run_qa.py) to [run_qa_adapter.py](https://github.com/piegu/language-models/blob/master/adapters/question-answering/run_qa_adapter.py) with the following changes:
- **EarlyStopping** by selecting the model with the highest eval f1 (patience of 3 before ending the training)
- **MAD-X 2.0** that allows not to train adapters in the last transformer layer (read page 6 of [UNKs Everywhere: Adapting Multilingual Language Models to New Scripts](https://arxiv.org/pdf/2012.15562.pdf))
- **Stack method** for the lang and task adapters when a lang adapter is loaded ([doc](https://docs.adapterhub.ml/adapter_composition.html?highlight=stack#stack))
- **Houlsby MHA last layer** that allows no to train the task adapter after the Feed Fordward but only after the MHA (Multi-Head Attention) in the last layer for the Houlsby configuration

## 2. Installation

In [1]:
import pathlib
from pathlib import Path

#root path
root = Path.cwd()

In [2]:
import pickle
import pandas as pd

In [3]:
import sys; print('python:',sys.version)

import torch; print('Pytorch:',torch.__version__)

import transformers; print('adapter-transformers:',transformers.__version__)
import transformers; print('HF transformers:',transformers.__hf_version__)
import tokenizers; print('tokenizers:',tokenizers.__version__)
import datasets; print('datasets:',datasets.__version__)

# import deepspeed; print('deepspeed:',deepspeed.__version__)

# Versions used in the virtuel environment of this notebook:

# python: 3.8.10 (default, Jun  4 2021, 15:09:15) 
# [GCC 7.5.0]
# Pytorch: 1.9.0
# adapter-transformers: 2.1.1
# HF transformers: 4.8.2
# tokenizers: 0.10.3
# datasets: 1.9.0

python: 3.8.10 (default, Jun  4 2021, 15:09:15) 
[GCC 7.5.0]
Pytorch: 1.9.0
adapter-transformers: 2.1.1
HF transformers: 4.8.2
tokenizers: 0.10.3
datasets: 1.9.0


Create symbolic links to the folder with the scripts to run or download them in the same folder of this notebook:

In [4]:
# ln -s ~/adapter-transformers/examples/question-answering/run_qa_adapter.py
# ln -s ~/adapter-transformers/examples/question-answering/trainer_qa.py
# ln -s ~/adapter-transformers/examples/question-answering/utils_qa.py

## 3. Model & dataset

In [5]:
# Select a MLM BERT base or large in the dataset language
# model_checkpoint = "neuralmind/bert-base-portuguese-cased"
model_checkpoint = "neuralmind/bert-large-portuguese-cased"

# We can upload the model from its local archive, too
model_checkpoint_original = "neuralmind/bert-large-portuguese-cased"
model_checkpoint_original_local = root.parent/'neuralmind-bert-large-portuguese-cased-lenerbr/mlm_base'

# SQuAD 1.1 in Portuguese
dataset_name = "squad11pt"

# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
version_2_with_negative = False # If true, some of the examples do not have an answer.

## 4. Main hyperparameters

In [6]:
task = "qa"

In [7]:
# training arguments
batch_size = 16
gradient_accumulation_steps = 1

learning_rate = 1e-4
num_train_epochs = 15.
early_stopping_patience = 5

adam_epsilon = 1e-7

fp16 = True
ds = False # If True, we use DeepSpeed

# best model
load_best_model_at_end = True 
metric_for_best_model = "f1"
greater_is_better = True

In [8]:
# train adapter
train_adapter = True # we want to train an adapter
load_adapter = None # we do not upload an existing adapter 

# lang adapter
with_adapters_mlm = False # if False, we do not upload an existing lang adapter

if with_adapters_mlm:
    adapter_composition = "stack" # we will stack the lang and task adapters
else:
    adapter_composition = None

# if True, do not put adapter in the last transformer layer (Pfeiffer configuration)
madx2 = False

# if True, put only an adapter after the MHA but not after the FF in the last layer (Houlsby configuration)
houlsby_MHA_lastlayer = True
if madx2:
    houlsby_MHA_lastlayer = False

## 5. Configuration

### GPU

In [9]:
# gpu
n_gpu = 1 # train on just one GPU
gpu = 0 # select the GPU

In [10]:
# Select GPU 0
import os
os.environ['MASTER_ADDR'] = 'localhost'
if gpu == 0:
    os.environ['MASTER_PORT'] = '9996' # modify if RuntimeError: Address already in use # GPU 0
elif gpu == 1:
    os.environ['MASTER_PORT'] = '9995'
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = str(gpu)
os.environ['WORLD_SIZE'] = "1"

### Adapters config

#### Task adapter

In [11]:
# task adapter config
adapter_config_name = "pfeiffer" # houlsby is possible, too
if adapter_config_name == "pfeiffer":
    adapter_non_linearity = 'gelu' # relu is possible, too
elif adapter_config_name == "houlsby":
    adapter_non_linearity = 'swish'
adapter_reduction_factor = 16
language = 'pt' # pt = Portuguese

#### Lang adapter

In [12]:
if with_adapters_mlm:
    
    # hyperparameters used for fine-tuning the MLM with lang adapter
    learning_rate_mlm = 1e-4
    batch_size_mlm = 16
    gradient_accumulation_steps_mlm = 1
    adam_epsilon_mlm = 1e-4
    num_train_epoch_mlm = 100.
    early_stopping_patience_mlm = 10
    madx2_mlm = madx2
    houlsby_MHA_lastlayer_mlm = houlsby_MHA_lastlayer
    ds_mlm = False
    fp16_mlm = True
    load_best_model_at_end_mlm = True
    metric_for_best_model_mlm = "loss"
    adapter_config_mlm = adapter_config_name + '+inv'

    # path to lang adapter
    outputs_mlm = model_checkpoint.replace('/','-') + '_' + dataset_name + '/' + 'mlm' + '/' \
    + 'lr' + str(learning_rate_mlm) \
    + '_bs' + str(batch_size_mlm) \
    + '_GAS' + str(gradient_accumulation_steps_mlm) \
    + '_eps' + str(adam_epsilon_mlm) \
    + '_epochs' + str(num_train_epoch_mlm) \
    + '_patience' + str(early_stopping_patience_mlm) \
    + '_madx2' + str(madx2_mlm) \
    + '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer_mlm) \
    + '_ds' + str(ds_mlm) \
    + '_fp16' + str(fp16_mlm) \
    + '_best' + str(load_best_model_at_end_mlm) \
    + '_metric' + str(metric_for_best_model_mlm) \
    + '_adapterconfig' + str(adapter_config_mlm)

    path_to_outputs_mlm = root/'outputs'/outputs_mlm
    
    # Config of the lang adapter
    lang_adapter_path = path_to_outputs_mlm/'adapters-mlm/'
    
    load_lang_adapter = str(lang_adapter_path)
    lang_adapter_config = str(lang_adapter_path) + "/adapter_config.json"
    if adapter_config_mlm == "pfeiffer+inv":
        lang_adapter_non_linearity = 'gelu' # relu is possible, too
    elif adapter_config_mlm == "houlsby+inv":
        lang_adapter_non_linearity = 'swish'
    lang_adapter_reduction_factor = 2
    language_mlm = language

### Training arguments of the HF trainer

In [13]:
# setup the training argument
do_train = True 
do_eval = True 

# if you want to test the trainer, set up the following variables
max_train_samples = 200 # None
max_val_samples = 50 # None

# epochs, bs, GA
evaluation_strategy = "epoch" 

# fp16
fp16_opt_level = 'O1'
fp16_backend = "auto"
fp16_full_eval = False

# optimizer (AdamW)
weight_decay = 0.01 # 0.0
adam_beta1 = 0.9
adam_beta2 = 0.999

# scheduler
lr_scheduler_type = 'linear'
warmup_ratio = 0.0
warmup_steps = 0

# logs
logging_strategy = "steps"
logging_first_step = True # False
logging_steps = 500     # if strategy = "steps"
eval_steps = logging_steps # logging_steps

# checkpoints
save_strategy = "epoch" # steps
save_steps = 500 # if save_strategy = "steps"
save_total_limit = 1 # None

# no cuda, seed
no_cuda = False
seed = 42

# bar
disable_tqdm = False # True
remove_unused_columns = True

In [14]:
# folder for training outputs

outputs = model_checkpoint.replace('/','-') + '_' + dataset_name

if with_adapters_mlm:
    outputs = outputs + '/' + 'mlm_' + str(task) + '_AdCompo' + str(adapter_composition) + '/'
else:
    outputs = outputs + '/' + str(task) + '/'

outputs = outputs \
+ 'lr' + str(learning_rate) \
+ '_bs' + str(batch_size) \
+ '_GAS' + str(gradient_accumulation_steps) \
+ '_eps' + str(adam_epsilon) \
+ '_epochs' + str(num_train_epochs) \
+ '_patience' + str(early_stopping_patience) \
+ '_wamlm' + str(with_adapters_mlm) \
+ '_madx2' + str(madx2) \
+ '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer) \
+ '_ds' + str(ds) \
+ '_fp16' + str(fp16) \
+ '_best' + str(load_best_model_at_end) \
+ '_metric' + str(metric_for_best_model) \
+ '_adapterconfig' + str(adapter_config_name)

# path to outputs
path_to_outputs = root/'outputs'/outputs

# subfolder for model outputs
output_dir = path_to_outputs/'output_dir' 
overwrite_output_dir = True # False

# logs
logging_dir = path_to_outputs/'logging_dir'

In [15]:
# The maximum total input sequence length after tokenization. Sequences longer
# than this will be truncated, sequences shorter will be padded.
max_seq_length = 384

# Whether to pad all samples to `max_seq_length`.
# If False, will pad the samples dynamically when batching to the maximum length in the batch (which can
# be faster on GPU but will be slower on TPU).
pad_to_max_length = True
    
# The threshold used to select the null answer: if the best answer has a score that is less than
# the score of the null answer minus this threshold, the null answer is selected for this example.
# Only useful when `version_2_with_negative=True`.
null_score_diff_threshold = 0.0

# When splitting up a long document into chunks, how much stride to take between chunks
doc_stride = 128
    
# The total number of n-best predictions to generate when looking for an answer.
n_best_size = 20
 
# The maximum length of an answer that can be generated. This is needed because the start
# and end predictions are not conditioned on one another.
max_answer_length = 30

## 6. Preparing the dataset

In [16]:
# %%time
# if dataset_name == "squad11pt":
    
#     # create dataset folder 
#     path_to_dataset = root/'data'/dataset_name
#     path_to_dataset.mkdir(parents=True, exist_ok=True) 

#     # Get dataset SQUAD in Portuguese
#     %cd {path_to_dataset}
#     !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn" -O squad-pt.tar.gz && rm -rf /tmp/cookies.txt

#     # unzip 
#     !tar -xvf squad-pt.tar.gz

#     # Get the train and validation json file in the HF script format 
#     # inspiration: file squad.py at https://github.com/huggingface/datasets/tree/master/datasets/squad

#     import json 
#     files = ['squad-train-v1.1.json','squad-dev-v1.1.json']

#     for file in files:

#         # Opening JSON file & returns JSON object as a dictionary 
#         f = open(file, encoding="utf-8") 
#         data = json.load(f) 

#         # Iterating through the json list 
#         entry_list = list()
#         id_list = list()

#         for row in data['data']: 
#             title = row['title']

#             for paragraph in row['paragraphs']:
#                 context = paragraph['context']

#                 for qa in paragraph['qas']:
#                     entry = {}

#                     qa_id = qa['id']
#                     question = qa['question']
#                     answers = qa['answers']

#                     entry['id'] = qa_id
#                     entry['title'] = title.strip()
#                     entry['context'] = context.strip()
#                     entry['question'] = question.strip()

#                     answer_starts = [answer["answer_start"] for answer in answers]
#                     answer_texts = [answer["text"].strip() for answer in answers]
#                     entry['answers'] = {}
#                     entry['answers']['answer_start'] = answer_starts
#                     entry['answers']['text'] = answer_texts

#                     entry_list.append(entry)

#         reverse_entry_list = entry_list[::-1]

#         # for entries with same id, keep only last one (corrected texts by the group Deep Learning Brasil)
#         unique_ids_list = list()
#         unique_entry_list = list()
#         for entry in reverse_entry_list:
#             qa_id = entry['id']
#             if qa_id not in unique_ids_list:
#                 unique_ids_list.append(qa_id)
#                 unique_entry_list.append(entry)

#         # Closing file 
#         f.close() 

#         new_dict = {}
#         new_dict['data'] = unique_entry_list

#         file_name = 'pt_' + str(file)
#         with open(file_name, 'w') as json_file:
#             json.dump(new_dict, json_file)
            
# %cd {root}

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [17]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

In [ ]:
from datasets import load_dataset, load_metric

if dataset_name == "squad11pt":
    
    # dataset folder 
    path_to_dataset = root/'data'/dataset_name
    
    # paths to files
    train_file = str(path_to_dataset/'pt_squad-train-v1.1.json')
    validation_file = str(path_to_dataset/'pt_squad-dev-v1.1.json')
    
    datasets = load_dataset('json', 
                            data_files={'train': train_file, \
                                        'validation': validation_file, \
                                       }, 
                            field='data')

To access an actual element, you need to select a split first, then give an index:

In [19]:
datasets["train"][10]

{'id': '5735c47ae853931400426b64',
 'title': 'Kathmandu',
 'context': 'A maioria das cozinhas encontradas em Katmandu não é vegetariana. No entanto, a prática do vegetarianismo não é incomum, e a culinária vegetariana pode ser encontrada em toda a cidade. O consumo de carne bovina é muito incomum e considerado tabu em muitos lugares. Buff (carne de búfalo Marinho) é muito comum. Há uma forte tradição de consumo de buffs em Katmandu, especialmente entre Newars, que não é encontrado em outras partes do Nepal. O consumo de carne de porco era considerado tabu até algumas décadas atrás. Devido à mistura com a cozinha Kirat do leste do Nepal, a carne de porco encontrou um lugar nos pratos de Katmandu. Uma população marginal de hindus e muçulmanos devotos o considera tabu. Os muçulmanos proíbem comer buff a partir do Alcorão, enquanto os hindus comem todas as variedades, exceto a carne de vaca, pois consideram a vaca uma deusa e símbolo da pureza. O café da manhã principal para moradores e vi

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [20]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [21]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,5731d9f3e17f3d1400422495,Bras%C3%ADlia,"Brasília (pronúncia em português: [bɾaˈziljɐ]) é a capital federal do Brasil e sede do governo do Distrito Federal. A cidade está localizada no topo do planalto brasileiro na região centro-oeste do país. Foi fundada em 21 de abril de 1960, para servir como a nova capital nacional. Brasília e seu metrô (abrangendo todo o Distrito Federal) tinham uma população de 2.556.149 em 2011, sendo a quarta cidade mais populosa do Brasil. Entre as principais cidades latino-americanas, Brasília possui o maior PIB per capita em R $ 61.915 (US $ 36.175).",Qual é a capital do Brasil?,"{'answer_start': [0], 'text': ['Brasília']}"
1,57278d02f1498d1400e8fbc9,Carnival,"Tarragona tem uma das sequências rituais mais completas da região. Os eventos começam com a construção de um enorme barril e terminam com a queima das efígies do rei e da rainha. No sábado, o desfile principal acontece com grupos mascarados, figuras zoomórficas, bandas de música e percussão e grupos com fogos de artifício (os demônios, o dragão, o boi, a fêmea). Grupos de carnaval se destacam por suas roupas cheias de elegância, mostrando exemplos brilhantes de artesanato em tecido, nos desfiles de sábado e domingo. Cerca de 5.000 pessoas são membros dos grupos de desfiles.",De que estão cheias as roupas dos grupos de carnaval?,"{'answer_start': [422], 'text': ['elegância']}"
2,5735190cacc1501500bac403,Hunting,"Desde 1934, a venda do Federal Duck Stamps gerou US $ 670 milhões e ajudou a comprar ou arrendar 5.200.000 acres (8.100 sq mi; 21.000 km2) de habitat. Os selos servem como uma licença para caçar aves migratórias, um passe de entrada para todas as áreas do Refúgio Nacional da Vida Selvagem, e também são considerados itens de colecionador, muitas vezes comprados por razões estéticas fora das comunidades de caça e observação de pássaros. Embora os não caçadores comprem um número significativo de selos de patos, oitenta e sete por cento de suas vendas são contribuídos por caçadores, o que é lógico, pois os caçadores são obrigados a comprá-los. A distribuição de fundos é gerenciada pela Comissão de Conservação de Aves Migratórias (MBCC).",O que os selos concedem ao comprador uma licença para fazer?,"{'answer_start': [189], 'text': ['caçar aves migratórias']}"
3,57278f77f1498d1400e8fc20,FA_Cup,"A possibilidade de vitórias improváveis nas rodadas anteriores da competição, em que equipes de classificação mais baixa vencem oposição mais alta, conhecida como ""matanças gigantes"", é muito esperada pelo público e é considerada parte integrante da tradição e prestígio da competição. , ao lado do ganho pelas equipes vencedoras da competição. Quase todos os clubes da Pirâmide da Liga têm um ato de matança de gigantes lembrado com carinho em sua história. É considerado particularmente digno de nota quando um time de primeira linha da Premier League sofre uma derrota chateada ou quando o assassino gigante é um clube que não pertence à liga, ou seja, de fora dos níveis profissionais da Liga de Futebol Americano.",O que é um assassino gigante?,"{'answer_start': [19], 'text': ['vitórias improváveis nas rodadas anteriores da competição, em que equipes de classificação mais baixa vencem oposição mais alta']}"
4,573254560fdd8d15006c69bd,Jehovah%27s_Witnesses,"Em agosto de 2015, as Testemunhas de Jeová relatam uma média de 8,2 milhões de editores - o termo que eles usam para os membros envolvidos ativamente na pregação - em 118.016 congregações. Em 2015, esses relatórios indicaram mais de 1,93 bilhões de horas gastas em atividades de pregação e ""estudo da Bíblia"". Desde meados dos anos 90, o número de publicadores de pico aumentou de 4,5 milhões para 8,2 milhões. No mesmo ano, eles conduziram ""estudos bíblicos"" com mais de 9,7 milhões de indivíduos, incluindo aqueles realizados pelos pais das Testemunhas de Jeová com seus filhos. As Testemunhas de Jeová estimam que a atual taxa de crescimento mundial é de 1,5% ao ano."

## 7. Training + Evaluation

#### Setup environment variables 

The magic command `%env` corresponds to `export` in linux. It allows to setup the values of all arguments of the script `run_qa_adapter.py`.

In [22]:
envs = {
'n_gpu':n_gpu,
'gpu':gpu,
'CUDA_VISIBLE_DEVICES':gpu,
'model_name_or_path':model_checkpoint,
'dataset_name':dataset_name,
'train_file':train_file,
'validation_file':validation_file,
'do_train':do_train,
'do_eval':do_eval,
'max_train_samples':max_train_samples,
'max_val_samples':max_train_samples,
'output_dir':output_dir,
'overwrite_output_dir':overwrite_output_dir,
'max_seq_length':max_seq_length,
'pad_to_max_length':pad_to_max_length,
'null_score_diff_threshold':null_score_diff_threshold,
'doc_stride':doc_stride,
'n_best_size':n_best_size,
'max_answer_length':max_answer_length,
'evaluation_strategy':evaluation_strategy,
'per_device_train_batch_size':batch_size,
'per_device_eval_batch_size':batch_size,
'gradient_accumulation_steps':gradient_accumulation_steps,
'learning_rate':learning_rate,
'weight_decay':weight_decay,
'adam_beta1':adam_beta1,
'adam_beta2':adam_beta2,
'adam_epsilon':adam_epsilon,
'num_train_epochs':num_train_epochs,
'warmup_ratio':warmup_ratio,
'warmup_steps':warmup_steps,
'logging_dir':logging_dir,
'logging_strategy':logging_strategy,
'logging_first_step':logging_first_step,
'logging_steps':logging_steps,
'eval_steps':eval_steps,
'save_strategy':save_strategy,
'save_steps':save_steps,
'save_total_limit':save_total_limit,
'no_cuda':no_cuda,
'seed':seed,
'fp16':fp16,
'fp16_opt_level':fp16_opt_level,
'fp16_backend':fp16_backend,
'fp16_full_eval':fp16_full_eval,
'disable_tqdm':disable_tqdm,
'remove_unused_columns':remove_unused_columns,
'load_best_model_at_end':load_best_model_at_end,
'metric_for_best_model':metric_for_best_model,
'greater_is_better':greater_is_better,
'early_stopping_patience':early_stopping_patience,
'madx2':madx2,
'houlsby_MHA_lastlayer':houlsby_MHA_lastlayer,
'train_adapter':train_adapter,
'adapter_config_name':adapter_config_name,
'adapter_non_linearity':adapter_non_linearity,
'adapter_reduction_factor':adapter_reduction_factor,
'language':language,
'adapter_composition':adapter_composition
}

if with_adapters_mlm:
    envs['load_lang_adapter']=load_lang_adapter
    envs['lang_adapter_config']=lang_adapter_config
    envs['lang_adapter_non_linearity']=lang_adapter_non_linearity
    envs['lang_adapter_reduction_factor']=lang_adapter_reduction_factor

In [23]:
for k,v in envs.items():
    %env {k}={v}

env: n_gpu=1
env: gpu=0
env: CUDA_VISIBLE_DEVICES=0
env: model_name_or_path=neuralmind/bert-base-portuguese-cased
env: dataset_name=squad11pt
env: train_file=/mnt/home/pierre/course-v4/nbs/MLM/language-modeling/data/squad11pt/pt_squad-train-v1.1.json
env: validation_file=/mnt/home/pierre/course-v4/nbs/MLM/language-modeling/data/squad11pt/pt_squad-dev-v1.1.json
env: do_train=True
env: do_eval=True
env: max_train_samples=200
env: max_val_samples=200
env: output_dir=/mnt/home/pierre/course-v4/nbs/MLM/language-modeling/models_outputs/neuralmind-bert-base-portuguese-cased_squad11pt/qa_lr0.0001_bs16_eps1e-06_epochs15.0_patience5_wamlmFalse_madx2False_houlsby_MHA_lastlayerTrue_dsFalse_fp16True_bestTrue_metricf1_adapterconfighoulsby/output_dir
env: overwrite_output_dir=True
env: max_seq_length=384
env: pad_to_max_length=True
env: null_score_diff_threshold=0.0
env: doc_stride=128
env: n_best_size=20
env: max_answer_length=30
env: evaluation_strategy=epoch
env: per_device_train_batch_size=16
env

#### Delete the output_dir (if exists)

In [24]:
!rm -r {output_dir}

Now, we can launch the training :-) 

In [ ]:
import os
PATH = os.getenv('PATH')
# replace xxxx by your username on your server (ex: paulo)
# replace yyyy by the name of the virtual environment of this notebook (ex: adapter-transformers)
%env PATH=/mnt/home/xxxx/anaconda3/envs/yyyy/bin:$PATH

In [27]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

In [25]:
# copy/paste/uncomment the 2 following lines in the following cell if you want to limit the number of data (useful for testing)
# --max_train_samples $max_train_samples \
# --max_val_samples $max_val_samples \

In [ ]:
%%time
if with_adapters_mlm:
    # with lang adapter
    !python -m torch.distributed.launch --nproc_per_node=$n_gpu run_qa_adapter.py \
    --model_name_or_path $model_name_or_path \
    --train_file $train_file \
    --validation_file $validation_file \
    --do_train $do_train \
    --do_eval $do_eval \
    --output_dir $output_dir \
    --overwrite_output_dir $overwrite_output_dir \
    --max_seq_length $max_seq_length \
    --pad_to_max_length $pad_to_max_length \
    --null_score_diff_threshold $null_score_diff_threshold \
    --doc_stride $doc_stride \
    --n_best_size $n_best_size \
    --max_answer_length $max_answer_length \
    --evaluation_strategy $evaluation_strategy \
    --per_device_train_batch_size $per_device_train_batch_size \
    --per_device_eval_batch_size $per_device_eval_batch_size \
    --gradient_accumulation_steps $gradient_accumulation_steps \
    --learning_rate $learning_rate \
    --weight_decay $weight_decay \
    --adam_beta1 $adam_beta1 \
    --adam_beta2 $adam_beta2 \
    --adam_epsilon $adam_epsilon \
    --num_train_epochs $num_train_epochs \
    --warmup_ratio $warmup_ratio \
    --warmup_steps $warmup_steps \
    --logging_dir $logging_dir \
    --logging_strategy $logging_strategy \
    --logging_first_step $logging_first_step \
    --logging_steps $logging_steps \
    --eval_steps $eval_steps \
    --save_strategy $save_strategy \
    --save_steps $save_steps \
    --save_total_limit $save_total_limit \
    --no_cuda $no_cuda \
    --seed $seed \
    --fp16 $fp16 \
    --fp16_opt_level $fp16_opt_level \
    --fp16_backend $fp16_backend \
    --fp16_full_eval $fp16_full_eval \
    --disable_tqdm $disable_tqdm \
    --remove_unused_columns $remove_unused_columns \
    --load_best_model_at_end $load_best_model_at_end \
    --metric_for_best_model $metric_for_best_model \
    --greater_is_better $greater_is_better \
    --early_stopping_patience $early_stopping_patience \
    --madx2 $madx2 \
    --houlsby_MHA_lastlayer $houlsby_MHA_lastlayer \
    --train_adapter $train_adapter \
    --adapter_config $adapter_config_name \
    --adapter_non_linearity $adapter_non_linearity \
    --adapter_reduction_factor $adapter_reduction_factor \
    --language $language \
    --adapter_composition $adapter_composition \
    --load_lang_adapter $load_lang_adapter \
    --lang_adapter_config $lang_adapter_config \
    --lang_adapter_non_linearity $lang_adapter_non_linearity \
    --lang_adapter_reduction_factor $lang_adapter_reduction_factor
else:
    # without lang adapter
    !python -m torch.distributed.launch --nproc_per_node=$n_gpu run_qa_adapter.py \
    --model_name_or_path $model_name_or_path \
    --train_file $train_file \
    --validation_file $validation_file \
    --do_train $do_train \
    --do_eval $do_eval \
    --output_dir $output_dir \
    --overwrite_output_dir $overwrite_output_dir \
    --max_seq_length $max_seq_length \
    --pad_to_max_length $pad_to_max_length \
    --null_score_diff_threshold $null_score_diff_threshold \
    --doc_stride $doc_stride \
    --n_best_size $n_best_size \
    --max_answer_length $max_answer_length \
    --evaluation_strategy $evaluation_strategy \
    --per_device_train_batch_size $per_device_train_batch_size \
    --per_device_eval_batch_size $per_device_eval_batch_size \
    --gradient_accumulation_steps $gradient_accumulation_steps \
    --learning_rate $learning_rate \
    --weight_decay $weight_decay \
    --adam_beta1 $adam_beta1 \
    --adam_beta2 $adam_beta2 \
    --adam_epsilon $adam_epsilon \
    --num_train_epochs $num_train_epochs \
    --warmup_ratio $warmup_ratio \
    --warmup_steps $warmup_steps \
    --logging_dir $logging_dir \
    --logging_strategy $logging_strategy \
    --logging_first_step $logging_first_step \
    --logging_steps $logging_steps \
    --eval_steps $eval_steps \
    --save_strategy $save_strategy \
    --save_steps $save_steps \
    --save_total_limit $save_total_limit \
    --no_cuda $no_cuda \
    --seed $seed \
    --fp16 $fp16 \
    --fp16_opt_level $fp16_opt_level \
    --fp16_backend $fp16_backend \
    --fp16_full_eval $fp16_full_eval \
    --disable_tqdm $disable_tqdm \
    --remove_unused_columns $remove_unused_columns \
    --load_best_model_at_end $load_best_model_at_end \
    --metric_for_best_model $metric_for_best_model \
    --greater_is_better $greater_is_better \
    --early_stopping_patience $early_stopping_patience \
    --madx2 $madx2 \
    --houlsby_MHA_lastlayer $houlsby_MHA_lastlayer \
    --train_adapter $train_adapter \
    --adapter_config $adapter_config_name \
    --adapter_non_linearity $adapter_non_linearity \
    --adapter_reduction_factor $adapter_reduction_factor \
    --language $language \
    --adapter_composition $adapter_composition

In [29]:
# folder of the saved task adapter
!ls -al {output_dir/task}

total 6756
drwxrwxr-x 2 pierre pierre    4096 Jul  9 23:52 .
drwxrwxr-x 4 pierre pierre    4096 Jul  9 23:56 ..
-rw-rw-r-- 1 pierre pierre     629 Jul  9 23:52 adapter_config.json
-rw-rw-r-- 1 pierre pierre     240 Jul  9 23:52 head_config.json
-rw-rw-r-- 1 pierre pierre 6889489 Jul  9 23:52 pytorch_adapter.bin
-rw-rw-r-- 1 pierre pierre    7143 Jul  9 23:52 pytorch_model_head.bin


In [ ]:
output_dir/task

Now, you can push the saved adapter + head to the [AdapterHub](https://adapterhub.ml/) (follow instructions at [Contributing to Adapter Hub](https://docs.adapterhub.ml/contributing.html)).

## 8. TensorBoard

In [30]:
#!pip install tensorboard

In [ ]:
import os
PATH = os.getenv('PATH')
# replace xxxx by your username on your server (ex: paulo)
# replace yyyy by the name of the virtual environment of this notebook (ex: adapter-transformers)
%env PATH=/mnt/home/xxxx/anaconda3/envs/yyyy/bin:$PATH

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir {logging_dir} --bind_all

## 9. Application QA

In [33]:
### import transformers
import pathlib
from pathlib import Path

In [34]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

if model_checkpoint_original == "neuralmind/bert-large-portuguese-cased":
    tokenizer_qa = AutoTokenizer.from_pretrained(model_checkpoint_original_local)
    model_qa = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint_original_local)
else:
    tokenizer_qa = AutoTokenizer.from_pretrained(model_checkpoint)
    model_qa = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at

### Lang adapter

In [ ]:
if with_adapters_mlm:
    
    # hyperparameters used for fine-tuning the MLM with lang adapter
    learning_rate_mlm = 1e-4
    batch_size_mlm = 16
    gradient_accumulation_steps_mlm = 1
    adam_epsilon_mlm = 1e-4
    num_train_epoch_mlm = 100.
    early_stopping_patience_mlm = 10
    madx2_mlm = madx2
    houlsby_MHA_lastlayer_mlm = houlsby_MHA_lastlayer
    ds_mlm = False
    fp16_mlm = True
    load_best_model_at_end_mlm = True
    metric_for_best_model_mlm = "loss"
    adapter_config_mlm = adapter_config_name + '+inv'

    # path to lang adapter
    outputs_mlm = model_checkpoint.replace('/','-') + '_' + dataset_name + '/' + 'mlm' + '/' \
    + 'lr' + str(learning_rate_mlm) \
    + '_bs' + str(batch_size_mlm) \
    + '_GAS' + str(gradient_accumulation_steps_mlm) \
    + '_eps' + str(adam_epsilon_mlm) \
    + '_epochs' + str(num_train_epoch_mlm) \
    + '_patience' + str(early_stopping_patience_mlm) \
    + '_madx2' + str(madx2_mlm) \
    + '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer_mlm) \
    + '_ds' + str(ds_mlm) \
    + '_fp16' + str(fp16_mlm) \
    + '_best' + str(load_best_model_at_end_mlm) \
    + '_metric' + str(metric_for_best_model_mlm) \
    + '_adapterconfig' + str(adapter_config_mlm)

    path_to_outputs_mlm = root/'outputs'/outputs_mlm
    
    # Config of the lang adapter
    lang_adapter_path = path_to_outputs_mlm/'adapters-mlm/'
    
    load_lang_adapter = str(lang_adapter_path)
    lang_adapter_config = str(lang_adapter_path) + "/adapter_config.json"

In [ ]:
if with_adapters_mlm:
    # load the language adapter without head
    task_mlm_load_as = 'mlm'
    lang_adapter_name = model_qa.load_adapter(
        load_lang_adapter,
        config=lang_adapter_config,
        load_as=task_mlm_load_as,
        with_head=False
        )

### QA adapter

In [ ]:
# folder for training outputs

outputs = model_checkpoint.replace('/','-') + '_' + dataset_name

if with_adapters_mlm:
    outputs = outputs + '/' + 'mlm_' + str(task) + '_AdCompo' + str(adapter_composition) + '/'
else:
    outputs = outputs + '/' + str(task) + '/'

outputs = outputs \
+ 'lr' + str(learning_rate) \
+ '_bs' + str(batch_size) \
+ '_GAS' + str(gradient_accumulation_steps) \
+ '_eps' + str(adam_epsilon) \
+ '_epochs' + str(num_train_epochs) \
+ '_patience' + str(early_stopping_patience) \
+ '_wamlm' + str(with_adapters_mlm) \
+ '_madx2' + str(madx2) \
+ '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer) \
+ '_ds' + str(ds) \
+ '_fp16' + str(fp16) \
+ '_best' + str(load_best_model_at_end) \
+ '_metric' + str(metric_for_best_model) \
+ '_adapterconfig' + str(adapter_config_name)

# path to outputs
path_to_outputs = root/'outputs'/outputs

# path to adapter
adapters_folder = 'adapters-' + task
path_to_save_adapter = path_to_outputs/adapters_folder

# Config of the task adapter
load_adapter = str(path_to_save_adapter)
adapter_config = str(path_to_save_adapter) + "/adapter_config.json"

In [ ]:
# load the task adapter with head
task_name = task
model_qa.load_adapter(
    load_adapter,
    config=adapter_config,
    load_as=task_name,
    with_head = True
)

### Active adapters

In [ ]:
# Set the adapters to be used in every forward pass
from transformers.adapters.composition import Stack
if lang_adapter_name:
    model_qa.active_adapters = Stack(task_mlm_load_as, task_name)
else:
    model_qa.set_active_adapters(task_name)

In [ ]:
model_qa

### Prediction

In [ ]:
from transformers import pipeline
nlp = pipeline("question-answering", model=model_qa, tokenizer=tokenizer_qa)

In [37]:
# source: https://pt.wikipedia.org/wiki/Pandemia_de_COVID-19
context = r"""A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19, 
uma doença respiratória causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2). 
O vírus tem origem zoonótica e o primeiro caso conhecido da doença remonta a dezembro de 2019 em Wuhan, na China. 
Em 20 de janeiro de 2020, a Organização Mundial da Saúde (OMS) classificou o surto 
como Emergência de Saúde Pública de Âmbito Internacional e, em 11 de março de 2020, como pandemia. 
Em 18 de junho de 2021, 177 349 274 casos foram confirmados em 192 países e territórios, 
com 3 840 181 mortes atribuídas à doença, tornando-se uma das pandemias mais mortais da história.
Os sintomas de COVID-19 são altamente variáveis, variando de nenhum a doenças com risco de morte. 
O vírus se espalha principalmente pelo ar quando as pessoas estão perto umas das outras. 
Ele deixa uma pessoa infectada quando ela respira, tosse, espirra ou fala e entra em outra pessoa pela boca, nariz ou olhos.
Ele também pode se espalhar através de superfícies contaminadas. 
As pessoas permanecem contagiosas por até duas semanas e podem espalhar o vírus mesmo se forem assintomáticas.
"""

In [ ]:
%%time
question = "Quando começou a pandemia de Covid-19 no mundo?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Qual é a data de início da pandemia Covid-19 em todo o mundo?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "A Covid-19 tem algo a ver com animais?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Onde foi descoberta a Covid-19?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Quantos casos houve?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Quantos mortes?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Quantos paises tiveram casos?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Quais são sintomas de COVID-19"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Como se espalha o vírus?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

# END